In [34]:
import numpy as np
import pprint
import sys
import os
import pdb
from typing import List, Union
if "../../" not in sys.path:
  sys.path.append("../../") 
from lib.envs.gridworld import GridworldEnv
# print = pprint.pprint

In [35]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [36]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        delta = 0
        for state in env.P:
            v = 0
            for action in env.P[state]:
                for p, next_s, r, done in env.P[state][action]:
                    v += policy[state][action]*p*(r + discount_factor*V[next_s])
            delta = max(delta, np.abs(v - V[state]))
            V[state] = v
        if delta < theta:
            break
    return np.array(V)

In [42]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    def list_action_value(value_function: list, state: int) -> list:
        A = np.zeros([env.nA])
        for action in env.P[state]:
            for prob, next_state, reward, done in env.P[state][action]:
                A[action] += prob * (reward + discount_factor*value_function[next_state])
        return A
    
    # Step 1: Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
#     pdb.set_trace()
    while True:
        policy_stable = True
        # Step 2: Policy evaluation
        V = policy_eval_fn(policy, env)
        
        # Step 3: Policy Iteration
        for state in env.P:
            old_best_action = np.argmax(policy[state])
            new_best_action = np.argmax(list_action_value(V, state))
            if old_best_action != new_best_action:
                policy_stable = False
            policy[state] = np.eye(env.nA)[new_best_action]
        if policy_stable: return policy, V

In [43]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

'Policy Probability Distribution:'
array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])
''
'Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):'
array([[0, 3, 3, 2],
       [0, 0, 0, 2],
       [0, 0, 1, 2],
       [0, 1, 1, 0]], dtype=int64)
''
'Value Function:'
array([ 0., -1., -2., -3., -1., -2., -3., -2., -2., -3., -2., -1., -3.,
       -2., -1.,  0.])
''
'Reshaped Grid Value Function:'
array([[ 0., -1., -2., -3.],
       [-1., -2., -3., -2.],
       [-2., -3., -2., -1.],
       [-3., -2., -1.,  0.]])
''


In [44]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)